In [3]:
import numpy as np
import matplotlib.pyplot as plt


import scipy.io as scio

In [4]:

def scale(X,center=True,scale=True):
    if(center==True):
        X =X -np.mean(X,axis=0)
    if(scale==True):
        X =X/np.std(X,axis=0)
    return X
    
def whiten(X):
    [N,p] =X.shape
    cov =(X.T@X)/N
    [eig_vals,eig_vecs] =np.linalg.eig(cov)
    P =(np.diag(1.0/np.sqrt(eig_vals)))@eig_vecs.T
    return [P,(P@X.T).T]

def group(y,B=500):
    N =y.shape[0]
    y =np.sort(y)
    y_min =y[0]
    y_max =y[-1]
    freqs =np.zeros((B,),dtype=float)
    ys    =np.zeros((B,),dtype=float)
    gaps =(y_max-y_min)/(B-1)
    left =y_min -0.5*gaps
    ys =y_min+gaps*np.arange(B)
    index =0
    for i in range(N):
        index =int(np.floor((y[i]-left)/gaps))
        freqs[index] = freqs[index]+1.0
    freqs =freqs/gaps
    return [ys,freqs]
        
        
    
    

def Amari_metric(A0,A):
    R =np.abs(A0@np.linalg.inv(A))
    m =A0.shape[0]
    row_max =np.max(R,axis=1)
    col_max =np.max(R,axis=0)
    return -1.0+(np.sum(np.sum(R,axis=1)/row_max)+np.sum(np.sum(R,axis=0)/col_max))/(2*m)
    
def mixmat(m):
    A =np.random.normal(size=(m,m))
    [u,s,vh] =np.linalg.svd(A,full_matrices=True)
    d =np.sort(np.random.uniform(low=0.0,high=1.0,size=(m,)))+1.0
    print('condition ',d[-1]/d[0])
    A =u@vh.T@np.diag(d)
    return A

def orth(W):
    [u,s,vh] =np.linalg.svd(W,full_matrices=True)
    W =u@vh
    return W


def gauss_kernel(x,knots,bw):
    n=x.shape[0]
    m=knots.shape[0]
    gram =np.zeros((n,m),dtype=float)
    for i in range(n):
        for j in range(m):
            gram[i,j]=np.exp(-1.0*bw*(x[i]-knots[j])*(x[i]-knots[j]))
    return gram

def poly_kernel(x,knots,s=1.0,d=3):
    n=x.shape[0]
    m=knots.shape[0]
    gram =np.zeros((n,m),dtype=float)
    for i in range(n):
        for j in range(m):
            gram[i,j]=np.power(s*x[i]*knots[j]+1,d)
    return gram

In [5]:
path =r"./data/dists.mat"
data = scio.loadmat(path)["dists"]

In [6]:
m  =2
S  =data[[11,7],[3,4],:].T
print('S shape ',S.shape)
A0 =mixmat(2)
X  =(A0@S.T).T

X  =scale(X,center=True,scale=False)
[P,X] =whiten(X)
target =np.linalg.inv(P@A0)
W0 =np.random.normal(size=(2,2))
W0 =orth(W0)
s= ((W0@(X.T)).T)[:,0]

S shape  (1024, 2)
condition  1.05350935257


In [8]:
N=1024
s =np.random.normal(size=(N,))
s =(s-np.mean(s))/np.std(s)

bws =np.linspace(0,10,1000)
for i in range(1000):
    gram =gauss_kernel(s,s,bws[i])
    if(np.linalg.matrix_rank(gram)==N):
        print("bw ",bws[i])

KeyboardInterrupt: 

In [44]:

print(np.linalg.matrix_rank(gram))

43


In [54]:
[eig_vals,eig_vecs] =np.linalg.eig(gram)

In [55]:
[eig_vals,eig_vecs] =np.linalg.eig(gram)
np.min(eig_vals)

-1.6065291424249355e-16

In [50]:
new_gram =eig_vecs@np.diag(eig_vals-np.min(eig_vals)+1e-12)@eig_vecs.T

In [52]:
[eig_vals,eig_vecs] =np.linalg.eig(new_gram)
print(np.min(eig_vals))
print(np.linalg.matrix_rank(new_gram))

1.00019557261e-12
50


In [56]:
np.linalg.norm(eig_vecs@np.diag(eig_vals)@eig_vecs.T-gram)

5.5116988007111949e-14